In [1]:
%load_ext autoreload
%autoreload 2

# Simple RAG (Retrieval-Augmented Generation) System
## Overview

This code implements a basic Retrieval-Augmented Generation (RAG) system for processing and querying PDF documents. The system encodes the document content into a vector store, which can then be queried to retrieve relevant information.

## Key Components

1. PDF processing and text extraction
2. Text chunking for manageable processing
3. Vector store creation using [FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/) and OpenAI embeddings
4. Retriever setup for querying the processed documents
5. Evaluation of the RAG system

### Import libraries and environment variables

In [2]:
import os
import sys

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/home/stackops/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
)

In [4]:
from helpers.helper_functions import *
from helpers.evalute_rag import *

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

/tmp/ipykernel_32842/1451311806.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from helpers.helper_functions import *


### Read Docs

In [5]:
path = "/home/stackops/langchain-labs/data/vks/pdf/vi/01-vks-la-gi.pdf"

### Encode document

In [6]:
def encode_pdf(path: str, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using OpenAI embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A FAISS vector store containing the encoded book content.
    """

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # Create embeddings (Tested with OpenAI and Amazon Bedrock)
    embeddings = get_langchain_embedding_provider(EmbeddingProvider.HUGGINGFACE)
    #embeddings = get_langchain_embedding_provider(EmbeddingProvider.AMAZON_BEDROCK)

    # Create vector store
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore

In [7]:
chunks_vector_store = encode_pdf(path, chunk_size=1000, chunk_overlap=200)

/home/stackops/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Create retriever

In [8]:
chunks_query_retriever = chunks_vector_store.as_retriever(search_kwargs={"k": 3})

### Test retriever

In [9]:
test_query = "VKS là gì vậy?"
context = retrieve_context_per_question(test_query, chunks_query_retriever)
show_context(context)

Context 1:
1. Giới thiệu về VNGCloud Kubernetes
Service - VKS
1.1. VNGCloud Kubernetes Service (VKS) là gì?
VNGCloud Kubernetes Service (hay còn gọi là VKS) là một dịch vụ quản lý Kubernetes được cung
cấp bởi VNGCloud. VKS giúp bạn triển khai và quản lý các ứng dụng dựa trên container một cách
dễ dàng và hiệu quả. VKS giúp bạn tập trung vào việc phát triển ứng dụng mà không cần quan tâm
đến việc quản lý Control Plane của Kubernetes.
VKS (VNGCloud Kubernetes Service) là một dịch vụ được quản lý trên VNGCloud giúp bạn đơn
giản hóa quá trình triển khai và quản lý các ứng dụng dựa trên container. Kubernetes là một nền
tảng mã nguồn mở được phát triển bởi Google, được sử dụng rộng rãi để quản lý và triển khai các
ứng dụng container trên môi trường phân tán.


Context 2:
1.2. Những điểm nổi bật của VKS
Những điểm nổi bật của dịch vụ VKS có thể kể đến gồm:
Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS sẽ giải phóng
bạn khỏi gánh nặng quản lý Control Plane của Kube

/home/stackops/langchain-labs/rag-techniques/helpers/helper_functions.py:90: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = chunks_query_retriever.get_relevant_documents(question)


### Evaluate results

In [10]:
#Note - this currently works with OPENAI only
evaluate_rag(chunks_query_retriever, 1)

{'questions': ['##  How might the changing climate impact the cultural practices and traditions of indigenous communities in the Arctic, and what steps can be taken to mitigate these impacts? ',
  ''],
 'results': ['```json\n{\n  "Relevance": 1,\n  "Completeness": 1,\n  "Conciseness": 1\n}\n```\n\n**Explanation:**\n\n* **Relevance:** The retrieved information is completely irrelevant to the question. It describes a cloud service called VNGCloud Kubernetes Service, which has nothing to do with climate change, indigenous communities, or cultural practices.\n* **Completeness:** The context is not complete in any way related to the question. It provides information about a cloud service, but it doesn\'t offer any insights into the impact of climate change on indigenous communities.\n* **Conciseness:** While the context is concise in describing the VNGCloud Kubernetes Service, it\'s irrelevant to the question, making conciseness a moot point. \n\n**Overall:** The retrieved context is comple